In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse

import torchvision.models as models
from tqdm import tqdm

In [11]:
import sys
sys.path.append('/content/drive/MyDrive/ece285_project/other_projects/SimCLR_v1/')
from models.resnet_simclr import ResNetSimCLR

In [3]:
data_dir = '/content/drive/MyDrive/ece285_project/DataSets'
dataset = 'cifar10'

lr = 0.01
n_epochs = 100

###for simclr
model_dir = '/content/drive/MyDrive/ece285_project/SimCLR/Runs/zig_zag_v2.2/checkpoint_0100.pth.tar'
model_loader = torch.load(model_dir)['state_dict']

###forSupervised

##SelfSuperVised
 

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Data
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
if dataset == 'cifar10':
    trainset = torchvision.datasets.CIFAR10(
        root=data_dir, train=True, download=True, transform=transform_train)
    trainloader = torch.utils.data.DataLoader(
        trainset, batch_size=128, shuffle=True, num_workers=2)

    testset = torchvision.datasets.CIFAR10(
        root=data_dir, train=False, download=True, transform=transform_test)
    testloader = torch.utils.data.DataLoader(
        testset, batch_size=100, shuffle=False, num_workers=2)

    classes = ('plane', 'car', 'bird', 'cat', 'deer',
              'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


Split the train set into 1% or 10%, with same class distribution


In [5]:
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split

label_fraction = 0.99
batch_size = 128
SEED = 42

# generate indices: instead of the actual data we pass in integers instead
train_indices, _, _, _ = train_test_split(
    range(len(trainset)),
    trainset.targets,
    stratify=trainset.targets,
    test_size=label_fraction,
    random_state=SEED
)

# generate subset based on indices
trainset_fraction = Subset(trainset, train_indices)

# create batches
trainloader = torch.utils.data.DataLoader(
        trainset_fraction, batch_size=batch_size, shuffle=True, num_workers=2)

In [6]:
len(trainset_fraction)

500

In [8]:
def train(model, epoch):
    print('\nEpoch: %d' % epoch)
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    acc = 100.*correct/total
    print('\Train Accuracy - %f' % acc)
    print('\Train Loss - %f' % train_loss)
    print('\Count is - %d'%total)

def test(model, epoch):
    global best_acc, state
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()


    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        state = {
            'net': model.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        # if not os.path.isdir(run_dir):
        #     os.mkdir(run_dir)
        best_acc = acc
    print('\Test Accuracy - %f' % acc)
    print('\Test Loss - %f' % test_loss)

###SimCLR

In [9]:
def append_dropout(model, rate=0.05):
      for name, module in model.named_children():
          if len(list(module.children())) > 0:
            append_dropout(module)
          if isinstance(module, nn.ReLU):
            new = nn.Sequential(module, nn.Dropout2d(p=rate, inplace=False))
            setattr(model, name, new)
          if isinstance(module, nn.Linear):
            new = nn.Sequential(module, nn.Dropout2d(p=0.2, inplace=False))
            setattr(model, name, new)

In [12]:
####Simclr
model = ResNetSimCLR('resnet50', 128)

    
model.load_state_dict(model_loader)
model = model.backbone
model.fc = nn.Linear(model.fc[2].in_features, 10)


# append_dropout(model)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
lr = 0.05
optimizer = optim.SGD(model.parameters(), lr=lr,
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9) #decay LR by 0.2 every 20 epochs

In [13]:
# model

In [14]:
n_epochs = 100
for epoch in tqdm(range(start_epoch, start_epoch+n_epochs)):
    train(model, epoch)
    test(model, epoch)
    scheduler.step()


  0%|          | 0/100 [00:00<?, ?it/s]


Epoch: 0
\Train Accuracy - 19.000000
\Train Loss - 8.664698
\Count is - 500


  1%|          | 1/100 [00:03<05:03,  3.07s/it]

\Test Accuracy - 35.620000
\Test Loss - 180.590587

Epoch: 1
\Train Accuracy - 38.200000
\Train Loss - 6.957305
\Count is - 500


  2%|▏         | 2/100 [00:05<04:31,  2.77s/it]

...

\Test Accuracy - 47.700000
\Test Loss - 289.787126

Epoch: 98
\Train Accuracy - 100.000000
\Train Loss - 0.021460
\Count is - 500


 99%|█████████▉| 99/100 [04:10<00:02,  2.55s/it]

\Test Accuracy - 47.780000
\Test Loss - 289.623226

Epoch: 99
\Train Accuracy - 100.000000
\Train Loss - 0.015872
\Count is - 500


100%|██████████| 100/100 [04:13<00:00,  2.53s/it]

\Test Accuracy - 47.880000
\Test Loss - 289.937794


In [ ]:
####Supervised
model = models.resnet50()
lr = 0.01
append_dropout(model)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr,
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9) #decay LR by 0.2 every 20 epochs

In [ ]:
n_epochs = 200
for epoch in tqdm(range(start_epoch, start_epoch+n_epochs)):
    train(model, epoch)
    test(model, epoch)
    scheduler.step()


  0%|          | 0/200 [00:00<?, ?it/s]


Epoch: 0


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1320: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


\Train Accuracy - 9.820000
\Train Loss - 225.080888
\Count is - 5000


  0%|          | 1/200 [00:04<15:11,  4.58s/it]

...

\Test Accuracy - 30.670000
\Test Loss - 210.338106

Epoch: 198
\Train Accuracy - 22.240000
\Train Loss - 122.069644
\Count is - 5000


100%|█████████▉| 199/200 [15:03<00:04,  4.55s/it]

\Test Accuracy - 30.200000
\Test Loss - 214.644289

Epoch: 199
\Train Accuracy - 21.980000
\Train Loss - 121.810140
\Count is - 5000


100%|██████████| 200/200 [15:08<00:00,  4.54s/it]

\Test Accuracy - 30.680000
\Test Loss - 204.701352
